In [12]:
from numpy import genfromtxt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from numpy import *
import matplotlib
import csv
from scipy import sparse
%pylab
%matplotlib inline
from matplotlib import pyplot as plt
import plotly
from plotly.graph_objs import *
from plotly import tools
plotly.tools.set_credentials_file(username='pujaarajan', api_key='7m16vvaw4j')
from scipy import sparse as sp
from sklearn import random_projection

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


#Import Data

In [5]:
description = genfromtxt('description.csv', delimiter=',')
graph  = genfromtxt('graph.csv', delimiter=',')
social_evolution = genfromtxt('social_and_evolution.csv', delimiter=',')

#CCA FUNCTION

In [23]:
def cca(XA,XB,d):
    XA_c=matrix(XA-average(XA,0))
    XB_c=matrix(XB-average(XB,0))
    sig11=transpose(XA_c)*XA_c
    sig12=transpose(XA_c)*XB_c
    sig22=transpose(XB_c)*XB_c
    sig21=transpose(XB_c)*XA_c
    lambdaA,wA=linalg.eigh(inv(sig11)*sig12*inv(sig22)*sig21)
    ind=argsort(lambdaA)[::-1]
    return XA_c*matrix(wA[:,ind[0:d]])

#CCA 1 ON DESCRIPTION AND SOCIAL EVOLUTION

In [14]:
m=1829
n=8000
X_dat=genfromtxt('description.csv',delimiter=',')
X_c1=sparse.coo_matrix((m,n))
X_c1.data=X_dat[:,2].astype(float64)
X_c1.row=X_dat[:,0].astype(int32)
X_c1.col=X_dat[:,1].astype(int32)
description_X=X_c1.todense()

In [15]:
cca1 = cca(social_evolution,description_X,2)

#CCA 2 ON CCA 1 AND GRAPH

In [28]:
m = 1829
n = 146983197
X_dat=genfromtxt('graph.csv',delimiter=',')
X_c2=sp.coo_matrix((m,n))
transformer = random_projection.SparseRandomProjection()
sparse_X = transformer.fit_transform(X_c2)

In [29]:
graph_X=sparse_X.todense()

In [32]:
cca1.shape

(1829, 2)

In [30]:
graph_X.shape

(1829, 6438)

In [31]:
cca1 = cca(cca1,graph_X,1)

LinAlgError: Singular matrix

#OTHER

In [59]:
cca1_3dim = cca(social_evolution,X,3)

In [88]:
data = go.Scatter3d(
    x=cca1_3dim[:,0],
    y=cca1_3dim[:,1],
    z=cca1_3dim[:,2],
)
fig = go.Figure(data=[data])
py.iplot(fig, filename='simple-3d-scatter')